# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропусков](#null)
    * [Замена типа данных](#type)
    * [Обработка дубликатов](#duplicates)
    * [Лемматизация](#lemma)
    * [Категоризация данных](#cotegory)
3. [Ответьте на вопросы](#qustion)
4. [Общий вывод](#conclusion)

## Шаг 1. Откройте файл с данными и изучите общую информацию<a id="start"></a> 

In [1]:
import pandas as pd
credit_scoring = pd.read_csv('/datasets/data.csv')

In [2]:
# вывод первых 5 строк таблицы data
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
# получение общей информации о данных в таблице data
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 сталбцов с различными типами данных:
* children - количество детей;
* days_employed - общий трудовой стаж в днях;
* dob_years - возраст;
* education - уровень образование (высшее, среднее, неоконченное высшее, начальное, ученая степень);
* education_id - идентификатор образования (0 - высшее, 1 - среднее, 2 - неоконченное высшее, 3 - начальное, 4 - ученая степень);
* family_status - семейное положение;
* family_status_id - идентификатор семейного положения ( 0 - женат / замужем, 1 - гражданский брак, 2 - вдовец / вдова, 3 - в разводе, 4 - не женат / не замужем);
* gender - пол;
* income_type - вид занятости (сотрудник, пенсионер, компаньон, госслужащий, безработный, предприниматель, студент, в декрете);
* debt - задолжность (0 - нет задолженности, 1 - есть задолженность);
* total_income - общий заработок;
* purpose - цель кредита.

Таблица приводит значения характеристик , описанных выше, платёжеспособности каждого клиента.

### Вывод

Проблемы обнаруженные при первом визуальном осмотре таблицы:
1. в столбце children и education есть отрицательные значения, что является ошибкой;
2. в столбце eduation значения приведены в разном регистре;
3. количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения;
4. некоторые столбцы имеют вещественный тип данных (float). 

Важными столбцами для установления влияния семейное положение, количество детей и уровень дохода клиента на факт погашения кредита в срок являются: children, family_status, total_income, purpose и debt.

## Шаг 2. Предобработка данных<a id="preprocessing"></a>

### Обработка пропусков<a id="null"></a> 

In [4]:
#выявление пропусков методом isnull и sum
credit_scoring.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения замечены в столбцах days_employed и total_income. Для заполнения пропусков необходимо найти медианное значение для days_employed и total_income.

In [5]:
# Медиана общего дохода
total_income_median = credit_scoring['total_income'].median()
total_income_median

145017.93753253992

Так как в столбце days_employed есть отрицательные значение, то для точного определения медианы возьмем её значения по модулю. Возьмем по модулю столбец children, где также были замечены отрицательные значения.

In [6]:
#Модуль столбца days_employed
credit_scoring['days_employed'] = list(map(abs, credit_scoring['days_employed']))
#Модуль столбца children
credit_scoring['children'] = list(map(abs, credit_scoring['children']))

In [7]:
# Медиана стажа работы
days_employed_median = credit_scoring['days_employed'].median()
days_employed_median

2194.220566878695

Заменим пропущенные значения в столбцах медианами

In [8]:
#Заменим пропущенные значений столбца days_employed медианой
credit_scoring['days_employed'] = credit_scoring['days_employed'].fillna(value=days_employed_median)
#Заменим пропущенные значений столбца total_income медианой
credit_scoring['total_income'] = credit_scoring['total_income'].fillna(value=total_income_median)

### Вывод

Пропущенные значения были обнаружены только в столбцах days_employed и total_income. Данные сталбцы имееют тип float, следовательно, пропушенные значения относятся к типу NaN. Причинами появления пропусков в данных могут быть отсутствие данных о клиенте, ошибкой при заполнении данных, повреждение данных и т.д. В нашем случае пропуски заполнялись медианными значениями стажа работы в днях и общего дохода. 

### Замена типа данных<a id="type"></a> 

Заменим типы данных столбцов days_employed и total_income с float на int

In [9]:
#Замена типа данных на int столбца days_employed
credit_scoring['days_employed'] = credit_scoring['days_employed'].astype('int')
#Замена типа данных на int столбца total_income
credit_scoring['total_income'] = credit_scoring['total_income'].astype('int')

### Вывод

Для изменения типа данных используется метод astype(), аргументом которого является требуемый тип данных ('int').

### Обработка дубликатов<a id="duplicates"></a> 

Для выявления всех дубликатов в таблице приведем все значения столбца education в нижний регистр.

In [10]:
#Перевод столбца education в нижний регистр
credit_scoring['education'] = credit_scoring['education'].str.lower()

In [11]:
#Сумма всех дубликатов в таблице
credit_scoring.duplicated().sum()

71

In [12]:
#Удаление всех дубликатов
credit_scoring = credit_scoring.drop_duplicates().reset_index(drop=True)

In [13]:
#Провека наличия дубликатов
credit_scoring.duplicated().sum()

0

### Вывод

Поиск дубликатов осуществляется методом duplicated(), а для подсчета их количества используется метод sum(). Перевод значений столбца education в нижний регистр увеличило количество дубликатов. При удалении дубликатов использовался метод drop_duplicates() в связке с методом reset_index() и аргументом drop=True для переиндексации значений таблицы. Причинами появления дубликатов в данных могут быть неправильное соединение данных из разных источников, ошибки или повторения при занесении информации.

### Лемматизация<a id="lemma"></a> 

Для проведения лемматизации целей необходимо сначала сохранить в отдельной переменной уникальные значения столбца целей.

In [14]:
#Уникальные значения столбца purpose
prupose_unique = credit_scoring['purpose'].unique()

In [15]:
#Лемматизация столбца purpose
m = Mystem()
lemmas = []
for row in prupose_unique:
    lemmas.extend(m.lemmatize(row))

pd.Series(lemmas).value_counts()

                  59
\n                38
покупка           10
недвижимость      10
образование        9
автомобиль         9
жилье              7
с                  5
на                 4
операция           4
свой               4
строительство      3
свадьба            3
получение          3
высокий            3
сделка             2
дополнительный     2
жилой              2
коммерческий       2
заниматься         2
для                2
подержать          1
собственный        1
сыграть            1
подержанный        1
приобретение       1
семья              1
ремонт             1
проведение         1
профильный         1
сдача              1
со                 1
dtype: int64

### Вывод

Лемматизация целей кредита проводилась с помошью библиотеки pymystem3. В качестве данных использовались уникальные значения столбца целей. Для проведения лемматизации каждой строки столбца целей кредита использовался цикл for. Также приведено число упаменаний лемматизированных слов.

### Категоризация данных<a id="cotegory"></a> 

Выполним категоризацию важных для нас столбцов, а иммено children, family_status_id, total_income и purpose

* **Категоризация по наличию детей**

Используем всего две категории характеризующие наличее и отсутсвие у заемщиков детей.

In [16]:
#Категоризация столбца children
def children_group(children):
    if children < 1:
        return 'без детей'
    return 'с детьми'
credit_scoring['children_group'] = credit_scoring['children'].apply(children_group)

#Подсчет числа клиентов для котегорий "с детьми" и "без детей"
credit_scoring['children_group'].value_counts()

без детей    14091
с детьми      7363
Name: children_group, dtype: int64

* **Категоризация по семейному положению**

Семейное положение можно упростить до двух категорий: женат / замужем и не женат / не замужем.

In [17]:
#Категоризация столбца family_status
def family_status_group(family_status_id):
    if family_status_id <= 1:
        return 'женат / замужем'
    return 'не женат / не замужем'
credit_scoring['family_status_group'] = credit_scoring['family_status_id'].apply(family_status_group)

#Подсчет числа клиентов для котегорий семейного положения
credit_scoring['family_status_group'].value_counts()

женат / замужем          16490
не женат / не замужем     4964
Name: family_status_group, dtype: int64

* **Категоризация по общему доходу**

Распределим значения общего дохода клиентов по трем категориям: 
1. низкий достаток (до 125 000)
2. средний достаток (от 125 000 до 190 000)
3. высокий достаток (более 190 000)

In [18]:
#Категоризация столбца total_income
def total_income_group(total_income):
    if total_income < 125000:
        return 'низкий достаток'
    if total_income < 190000:
        return 'средний достаток'
    return 'высокий достаток'
credit_scoring['total_income_group'] = credit_scoring['total_income'].apply(total_income_group)

#Подсчет числа клиентов для котегорий семейного положения
credit_scoring['total_income_group'].value_counts()

средний достаток    8335
низкий достаток     7369
высокий достаток    5750
Name: total_income_group, dtype: int64

* **Категоризация по цели**

Категории целей выбраны с учетом лемматизации прозведенной выше. Всего 4 категории: недвижимость, образование, свадьба и автомобиль.

In [19]:
#Категоризация столбца purpose
def purpose_group(purpose):
    if 'недвижимост' in purpose:
        return 'недвижимость'
    if 'жил' in purpose:
        return 'недвижимость'
    if 'образован' in purpose:
        return 'образование'
    if 'свадьб' in purpose:
        return 'свадьба'
    if 'автомоб' in purpose:
        return 'автомобиль'
credit_scoring['purpose_group'] = credit_scoring['purpose'].apply(purpose_group)


### Вывод

Проведена категоризация важных для дальнейшего шага столбцов данных. Категоризация осуществлялось при помощи функции одной строки, где устанавливались условия для распределения определенных значений в нужную категорию. Для подачи на вход функции строк определенного столбца использовался метод apply().

## Шаг 3. Ответьте на вопросы<a id="qustion"></a> 

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Методом pivot_table() произведём подсчет числа людей с долгом и без для каждой категории.

In [20]:
#Подсчет числа людей с долгом и без для каждой котегории
children_vs_debt = credit_scoring.pivot_table(index=['children_group'], 
    columns= 'debt', values='children', aggfunc= 'count')
children_vs_debt

debt,0,1
children_group,,
без детей,13028,1063
с детьми,6685,678


Для определения зависимости наличия детей на возврат кредита просчитаем долю задолжников каждой категории от общего числа заёмщиков в процентах.

In [21]:
#Посчитаем общее число заемщиков с детьми и без детей
total_children = credit_scoring.groupby('children_group')['debt'].count()
total_children

children_group
без детей    14091
с детьми      7363
Name: debt, dtype: int64

In [22]:
#Доля задолжников от общего числа заёмщиков с детьми и без детей в процентах 
part_children = children_vs_debt[1]/total_children*100
part_children

children_group
без детей    7.543822
с детьми     9.208203
dtype: float64

### Вывод

Из результатов исследования зависимости между наличием детей и возвратом кредита в срок следует, что доля должников с детьми примерно на 2% больше числа задолжников без детей, т.е. задолженность заёмщиков зависит от наличия детей.

In [36]:
# Код ревьюера

credit_scoring.pivot_table(index=['children_group'], 
    values='debt', aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,debt,debt,debt
children_group,,,
без детей,14091,1063,0.075438
с детьми,7363,678,0.092082


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
#Подсчет числа людей с долгом и без для каждой котегории
family_status_vs_debt = credit_scoring.pivot_table(index=['family_status_group'], 
    columns= 'debt', values='family_status', aggfunc= 'count')
family_status_vs_debt

debt,0,1
family_status_group,,
женат / замужем,15171,1319
не женат / не замужем,4542,422


In [24]:
#Посчитаем общее число заемщиков по категориям семейного положения
total_family_status = credit_scoring.groupby('family_status_group')['debt'].count()
total_family_status

family_status_group
женат / замужем          16490
не женат / не замужем     4964
Name: debt, dtype: int64

In [25]:
#Доля задолжников от общего числа заёмщиков по категориям семейного положения в процентах 
part_family_status = family_status_vs_debt[1]/total_family_status*100
part_family_status

family_status_group
женат / замужем          7.998787
не женат / не замужем    8.501209
dtype: float64

### Вывод

По выведенным данным видно, что процент не женатых / не замужних немногим превышает процент женатых / замужних должников, следовательно, не возвратом кредита в срок чуть больше подвержены не женатые / не замужние люди.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
#Подсчет числа людей с долгом и без для каждой котегории
total_income_vs_debt = credit_scoring.pivot_table(index=['total_income_group'], 
    columns= 'debt', values='total_income', aggfunc= 'count')
total_income_vs_debt

debt,0,1
total_income_group,,
высокий достаток,5332,418
низкий достаток,6761,608
средний достаток,7620,715


In [27]:
#Посчитаем общее число заемщиков по категориям семейного положения
total_total_income = credit_scoring.groupby('total_income_group')['debt'].count()
total_total_income

total_income_group
высокий достаток    5750
низкий достаток     7369
средний достаток    8335
Name: debt, dtype: int64

In [28]:
#Доля задолжников от общего числа заёмщиков по категориям семейного положения в процентах 
part_total_income = total_income_vs_debt[1]/total_total_income*100
part_total_income

total_income_group
высокий достаток    7.269565
низкий достаток     8.250780
средний достаток    8.578284
dtype: float64

### Вывод

По результатам видно, что лиди с высоким достатком гасят кредит чаще, чем люди со средним и низким достатком. Также процент людей с низким и средним достатком примерно равен.

- Как разные цели кредита влияют на его возврат в срок?

In [29]:
#Подсчет числа людей с долгом и без для каждой котегории
purpose_vs_debt = credit_scoring.pivot_table(index=['purpose_group'], 
    columns= 'debt', values='purpose', aggfunc= 'count')
purpose_vs_debt

debt,0,1
purpose_group,,
автомобиль,3903,403
недвижимость,10029,782
образование,3643,370
свадьба,2138,186


In [30]:
#Посчитаем общее число заемщиков по категориям семейного положения
total_purpose = credit_scoring.groupby('purpose_group')['debt'].count()
total_purpose

purpose_group
автомобиль       4306
недвижимость    10811
образование      4013
свадьба          2324
Name: debt, dtype: int64

In [31]:
#Доля задолжников от общего числа заёмщиков по категориям семейного положения в процентах 
part_purpose = purpose_vs_debt[1]/total_purpose*100
part_purpose

purpose_group
автомобиль      9.359034
недвижимость    7.233373
образование     9.220035
свадьба         8.003442
dtype: float64

### Вывод

По результатам исследования влияния разных целей на задолженность можно утверждать, что наибольшая доля должников среди людей выбравших в качестве цели кредита автомобиль и образование (9.4% и 9.2% соответственно), что на 1% больше доли должников потративших кредит на свадебные нужды (8.0%) и на 2% больше людей выбравших целью кредита недвижимость (7.2%).

## Шаг 4. Общий вывод<a id="conclusion"></a> 

В ходе выполнения проекта было произведено исследование надёжности заёмщиков и выявлено влияние наличия детей, семейного положения, уровня дохода и различных целей клиента на погашение кредита в срок.

Перед проведением исследования данных, также было произведено первичное изучение файла, в ходе которого выявлены пропуски, отрицательные значения, различия типов данных и различия регистра в написании слов. При дальнейшей предобработки данных осуществлены следующие действия:
* выявлены пропуски данных в столбцах days_employed и total_income и заменены их медианами;
* значения типов данных столбцов days_employed и total_income переведены из float в int;
* обнаружены и удалены все дубликаты;
* произведена лемматизация целей кредита;
* выполнена категоризация данных по установленным условиям распределения важных для исследования столбцов (children, family_status_id, total_income и purpose).

При проведении исследований сделаны следующие выводы:
1. Наличие детей у заёмщика влияет на погашения кредита в срок. Доля должников с детьми - 7.5%, без детей - 9.2%;
2. Статус семейного положения незначительно влияет на факт задолженности. Доля должников женатых / замужних - 8.0%, не женатых / не замужемних - 8.5%;
3. Существует зависимость низкого и среднего уровня дохода по отношению к высокому уровню дохода и возврата кредита в срок. Между низким и средним уровнем дохода зависимости нет. Доля должников с высоким достатком - 7.3%, со средним достатком - 8.6%, с низким достатком - 8.3%;
4. Различные цели получения кредита также влияют на его погашение. Доля должников с целью автомобиль - 9.4%, образование - 9.2%, свадьба - свадьба 8.0% и недвижимость - 7.2%.